In [5]:
import gurobipy as grb
import pandas as pd


categories = {
    "good":100,
    "bad":10,
    "terrible":1
}

multipliers = {
    (0,0,0):1,
    (1,0,0):1,
    (0,1,0):10,
    (0,0,1):100,
    (2,0,0):1,
    (1,1,0):10,
    (1,0,1):2500,
    (0,2,0):1000,
    (0,1,1):10000,
    (0,0,2):100000,
    (3,0,0):1,
    (2,1,0):10,
    (2,0,1):1000,
    (1,2,0):500,
    (1,1,1):1000,
    (1,0,2):100000,
    (0,3,0):5000,
    (0,2,1):10000,
    (0,1,2):100000,
    (1,1,2):100000,
    (0,0,3):100000
}

venues_all = {
    "A301":"good",
    "A302":"good",
    "A303":"terrible",
    "A304":"terrible",
    "A305":"good",
    "A310":"terrible",
    "A311":"good",
    "A313":"good",
    "A315":"good",
    "A401":"good",
    "A402":"good",
    "A404":"good",
    "A406":"good",
    "A408":"good",
    "A410":"good",
    "A412a":"good"
    
}


venues = {}
venues[1] = {
    "A301":50,
    "A302":100,
    "A303":1,
    "A304":1,
    "A305":100,
    "A310":1,
    "A311":50,
    "A313":50
}

venues[2] = {
    "A315":50,
    "A401":50,
    "A402":50,
    "A404":100,
    "A406":50,
    "A408":50,
    "A410":100,
    "A412a":100
}


def allocate(rd):
    current = rd
    past = rd-1
    
    data = pd.read_csv("data_afterr{}.csv".format(past),index_col=0)#.format(past),index_col=0)
    
    teams_records = {}
    teams_multipliers = {}
    
    for ind,row in data.iterrows():
        good = 0
        bad = 0
        terrible = 0
        
        for rd_col in (1,2,3):
            if row[rd_col] == "--":
                pass
            elif row[rd_col] == "nan":
                pass
            elif venues_all[row[rd_col]] == "good":
                good += 1
            elif venues_all[row[rd_col]] == "bad":
                bad += 1
            elif venues_all[row[rd_col]] == "terrible":
                terrible += 1
        teams_records[ind] = [good,bad,terrible]
        
        # Add the highest multiplier to Jonesboro 1/2 and AIS so that they get large rooms regardless
        if ind == "McCollum":
            teams_multipliers[ind] = 100000
        elif ind == "Tucker":
            teams_multipliers[ind] = 100000
        elif ind == "Baldwin":
            teams_multipliers[ind] = 100000
        else:
            teams_multipliers[ind] = multipliers[(good,bad,terrible)]
    
    print(teams_multipliers)
    
    pairings = {}
    pairingsd1_csv = pd.read_csv("pairings_r{}_div1.csv".format(current),index_col="rank")
    pairingsd2_csv = pd.read_csv("pairings_r{}_div2.csv".format(current),index_col="rank")
    
    
    pairings[1] = {}
    pairings[2] = {}
    for ind,row in pairingsd1_csv.iterrows():
        pairings[1][ind] = [row[0],row[1]]
    for ind,row in pairingsd2_csv.iterrows():
        pairings[2][ind] = [row[0],row[1]]

        
    costs = {}
    
    costs[1] = {}
    costs[2] = {}
    
    for pairingkey,pairingvalue in pairings[1].iteritems():
        pairing_cost = 0
        for team in pairingvalue:
            pairing_cost += teams_multipliers[team]
            
        for venue,cost in venues[1].iteritems():
            combined_cost = pairing_cost * cost
            costs[1][pairingkey,venue] = combined_cost
            
    for pairingkey,pairingvalue in pairings[2].iteritems():
        pairing_cost = 0
        for team in pairingvalue:
            pairing_cost += teams_multipliers[team]
            
        for venue,cost in venues[2].iteritems():
            combined_cost = pairing_cost * cost
            costs[2][pairingkey,venue] = combined_cost
        
            
    
    m1 = grb.Model("Division 1 Allocation")
    x1 = {}
    
    for key,value in costs[1].iteritems():
        x1[key] = m1.addVar(vtype = grb.GRB.BINARY, obj = value, name="x1_{}".format(key))
            
    #Each venue gets one pairing
    for key,value in venues[1].iteritems():
        m1.addConstr(sum(x1[pairing,key] for pairing in pairings[1].keys()) == 1)
    #Each pairing gets one venue
    for key,value in pairings[1].iteritems():
        m1.addConstr(sum(x1[key,venue] for venue in venues[1].keys()) == 1)
        
    m1.modelSense = grb.GRB.MAXIMIZE
    
    m1.optimize()    
            
    m2 = grb.Model("Division 2 Allocation")
    x2 = {}
    
    for key,value in costs[2].iteritems():
        x2[key] = m2.addVar(vtype = grb.GRB.BINARY, obj = value, name="x2_{}".format(key))
            
    #Each venue gets one pairing
    for key,value in venues[2].iteritems():
        m2.addConstr(sum(x2[pairing,key] for pairing in pairings[2].keys()) == 1)
    #Each pairing gets one venue
    for key,value in pairings[2].iteritems():
        m2.addConstr(sum(x2[key,venue] for venue in venues[2].keys()) == 1)
        
    m2.modelSense = grb.GRB.MAXIMIZE
    
    m2.optimize()
    
    
    
    print("-----")
    print("DIVISION 1 ALLOCATION - ROUND {}".format(current))
    for key,value in costs[1].iteritems():
        if x1[key].x != 0:
            print("Rank {} ({} v {}) in Room {}").format(key[0],pairings[1][key[0]][0],pairings[1][key[0]][1],key[1])
     
    print("-----")
    

    print("DIVISION 2 ALLOCATION - ROUND {}".format(current))
    for key,value in costs[2].iteritems():
        if x2[key].x != 0:
            print("Rank {} ({} v {}) in Room {}").format(key[0],pairings[2][key[0]][0],pairings[2][key[0]][1],key[1])

In [6]:
allocate(2)

{'Massey': 1, 'Helms': 1, 'McClain': 1, 'West': 1, 'Acres': 1, 'Moretz': 100, 'Hill': 100, 'Thompson': 1, 'Bentz': 100, 'Benjamin': 1, 'Pardue': 1, 'Lee': 1, 'Conroy': 1, 'Mitchell': 1, 'McCollum': 100000, 'Riggs': 100, 'Tucker': 100000, 'Pearman': 1, 'Reed': 1, 'Ordway': 1, 'King Jr': 1, 'Hill2': 1, 'Blomberg': 1, 'Berry': 100, 'Rowland': 1, 'Boykin': 1, 'Beckham': 1, 'Baldwin': 1, 'Newton': 1, 'Kelley': 1, 'Seacrest': 1, 'Nelson': 1}
Optimize a model with 16 rows, 64 columns and 128 nonzeros
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.004021e+07
Presolve removed 0 rows and 5 columns
Presolve time: 0.01s
Presolved: 16 rows, 59 columns, 118 nonzeros
Variable types: 0 continuous, 59 integer (59 binary)

Root relaxation: cutoff, 23 iterations, 0.00 seconds

    Nodes    |    Current N